In [1]:
# Gensim
import gensim
import gensim.corpora as corpora

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  

import ast



In [2]:
with open("processedtext.txt", "r") as processed:
    text = processed.read()

processed = ast.literal_eval(text)

# Print the converted list
print(processed)


['culture', 'disposable', 'gone', 'far', 'began', 'small', 'thing', 'plastic', 'spoon', 'used', 'place', 'metal', 'one', 'styrofoam', 'container', 'find', 'time', 'wash', 'dish', 'piece', 'tissue', 'carelessly', 'throw', 'away', 'moved', 'bigger', 'thing', 'like', 'carbon', 'emission', 'nuclear', 'waste', 'advancement', 'rocket', 'technology', 'even', 'planning', 'throw', 'whole', 'planet', 'away', 'unfortunately', 'merely', 'symptomatic', 'people', 'inadequate', 'concern', 'green', 'agenda', 'term', 'broadly', 'refers', 'political', 'movement', 'promoting', 'environmental', 'protection', 'sustainable', 'development', 'government', 'pay', 'lipservice', 'environmental', 'movement', 'placate', 'people', 'country', 'greater', 'risk', 'suffering', 'climate', 'change', 'taking', 'concrete', 'action', 'firm', 'put', 'thought', 'developing', 'sustainable', 'business', 'practice', 'still', 'subordinate', 'pursuit', 'profit', 'individual', 'level', 'hype', 'around', 'disposable', 'container', '

In [3]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(processed, min_count=5, threshold=100) # higher threshold fewer phrases.

bigram_mod = gensim.models.phrases.Phraser(bigram)


In [4]:
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]


In [5]:
bigrams = make_bigrams([processed])
bigrams

[['culture',
  'disposable',
  'gone',
  'far',
  'began',
  'small',
  'thing',
  'plastic',
  'spoon',
  'used',
  'place',
  'metal',
  'one',
  'styrofoam',
  'container',
  'find',
  'time',
  'wash',
  'dish',
  'piece',
  'tissue',
  'carelessly',
  'throw',
  'away',
  'moved',
  'bigger',
  'thing',
  'like',
  'carbon',
  'emission',
  'nuclear',
  'waste',
  'advancement',
  'rocket',
  'technology',
  'even',
  'planning',
  'throw',
  'whole',
  'planet',
  'away',
  'unfortunately',
  'merely',
  'symptomatic',
  'people',
  'inadequate',
  'concern',
  'green',
  'agenda',
  'term',
  'broadly',
  'refers',
  'political',
  'movement',
  'promoting',
  'environmental',
  'protection',
  'sustainable',
  'development',
  'government',
  'pay',
  'lipservice',
  'environmental',
  'movement',
  'placate',
  'people',
  'country',
  'greater',
  'risk',
  'suffering',
  'climate',
  'change',
  'taking',
  'concrete',
  'action',
  'firm',
  'put',
  'thought',
  'developin

In [6]:
id2word = corpora.Dictionary(bigrams)

# Create Corpus
texts = bigrams

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]


# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 3), (7, 1), (8, 1), (9, 2), (10, 1), (11, 3), (12, 2), (13, 1), (14, 1), (15, 1), (16, 1), (17, 20), (18, 4), (19, 1), (20, 2), (21, 1), (22, 2), (23, 1), (24, 1), (25, 1), (26, 1), (27, 2), (28, 2), (29, 1), (30, 2), (31, 1), (32, 2), (33, 1), (34, 2), (35, 1), (36, 1), (37, 2), (38, 1), (39, 1), (40, 7), (41, 1), (42, 1), (43, 1), (44, 1), (45, 2), (46, 1), (47, 1), (48, 1), (49, 3), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 6), (56, 1), (57, 4), (58, 3), (59, 6), (60, 2), (61, 2), (62, 3), (63, 1), (64, 5), (65, 1), (66, 3), (67, 1), (68, 1), (69, 4), (70, 2), (71, 5), (72, 1), (73, 2), (74, 1), (75, 1), (76, 2), (77, 1), (78, 1), (79, 2), (80, 13), (81, 1), (82, 1), (83, 2), (84, 1), (85, 1), (86, 1), (87, 2), (88, 2), (89, 1), (90, 1), (91, 1), (92, 1), (93, 1), (94, 1), (95, 3), (96, 2), (97, 1), (98, 5), (99, 1), (100, 1), (101, 1), (102, 1), (103, 3), (104, 1), (105, 2), (106, 2), (107, 1), (108, 1), (109, 1), (110, 

In [7]:
id2word[0]
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]


[[('1000', 1),
  ('14', 1),
  ('20', 1),
  ('2015', 1),
  ('2019', 1),
  ('2020', 1),
  ('2021', 3),
  ('60', 1),
  ('abiding', 1),
  ('able', 2),
  ('acceptance', 1),
  ('account', 3),
  ('action', 2),
  ('advancement', 1),
  ('adversely', 1),
  ('affect', 1),
  ('affected', 1),
  ('agenda', 20),
  ('agreement', 4),
  ('along', 1),
  ('also', 2),
  ('amazon', 1),
  ('america', 2),
  ('another', 1),
  ('appears', 1),
  ('around', 1),
  ('aside', 1),
  ('aspect', 2),
  ('attention', 2),
  ('authority', 1),
  ('away', 2),
  ('bamboo', 1),
  ('bare', 2),
  ('basel', 1),
  ('become', 2),
  ('becoming', 1),
  ('began', 1),
  ('behind', 2),
  ('benefit', 1),
  ('best', 1),
  ('beyond', 7),
  ('bigger', 1),
  ('bout', 1),
  ('broadly', 1),
  ('brushed', 1),
  ('business', 2),
  ('campaign', 1),
  ('capital', 1),
  ('car', 1),
  ('carbon', 3),
  ('care', 1),
  ('carelessly', 1),
  ('carried', 1),
  ('catching', 1),
  ('chance', 1),
  ('change', 6),
  ('chemical', 1),
  ('city', 4),
  ('clear',

In [8]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=2, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)



In [9]:
doc_lda = lda_model[corpus]

for idx, topic in lda_model.print_topics(-1):
    print("Topic: {} \nWords: {}".format(idx, topic ))
    print("\n")

Topic: 0 
Words: 0.023*"green" + 0.022*"agenda" + 0.014*"country" + 0.013*"government" + 0.008*"individual" + 0.008*"beyond" + 0.008*"environmental" + 0.008*"firm" + 0.007*"climate" + 0.007*"one"


Topic: 1 
Words: 0.003*"green" + 0.003*"agenda" + 0.003*"government" + 0.003*"country" + 0.002*"firm" + 0.002*"environmental" + 0.002*"change" + 0.002*"beyond" + 0.002*"fire" + 0.002*"taking"




In [10]:
pyLDAvis.enable_notebook()

vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds='mmds')   
# vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.001014 -0.017901       1        1  99.855009
1      0.001014  0.017901       2        1   0.144991, topic_info=              Term       Freq      Total Category  logprob  loglift
146     government   9.000000   9.000000  Default  30.0000  30.0000
80         country  10.000000  10.000000  Default  29.0000  29.0000
127           firm   5.000000   5.000000  Default  28.0000  28.0000
110  environmental   5.000000   5.000000  Default  27.0000  27.0000
40          beyond   5.000000   5.000000  Default  26.0000  26.0000
..             ...        ...        ...      ...      ...      ...
116        example   0.002562   3.493024   Topic2  -6.0304  -0.6813
103         effort   0.002556   2.715988   Topic2  -6.0330  -0.4323
345          small   0.002562   3.493128   Topic2  -6.0307  -0.6816
234       movement   0.002561   3.493214   Topic2  -6.0309  -0.6819
69   consideration   0.002561   3.493243   Topic2  -6.0310  -0.6819

[122 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
0         1  0.858087        1000
17        1  1.005654      agenda
18        1  0.859128   agreement
25        1  0.857707      around
26        1  0.857708       aside
...     ...       ...         ...
382       1  0.937424      taking
396       1  1.104733        true
397       1  0.857768       truly
398       1  0.857747  ultimately
418       1  1.104593       would

[65 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2])